In [ ]:
import spacy 
from spacy.pipeline import EntityRuler
from spacy import displacy
import pandas as pd
import numpy as np 


In [2]:
df = pd.read_csv("../data/sample_resumes/Resume/resumeDataSet2_transformed_filtered.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:200,
]
data.head()


,ID,Resume_str,Resume_html,Category
160,604,Core skills â¢ Project / Program Management â...,NaN,DevOps Engineer
312,853,Hobbies â¢ Playing Chess â¢ Solving Rubik's ...,NaN,Blockchain
215,714,Education Details \r\nJanuary 2016 BSc. Mumba...,NaN,Database
95,443,Education Details \r\n\r\nSAP Technical Archit...,NaN,SAP Developer
27,347,Skills â¢ Language: Java â¢ Operating System...,NaN,Java Developer


In [ ]:
# import en_core_web_lg


# nlp = spacy.blank("en")
nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("ner")

ruler = nlp.add_pipe("entity_ruler")

# ner = nlp.add_pipe("ner")

#Skills for entities 

SweSkills = ["Python", "Java", "JavaScript", "C++", "React", "Angular", "Node.js", "Git", "CI/CD"]
DsSkills = ["Python", "R", "SQL", "Machine Learning", "Data Analysis", "Statistics", "TensorFlow", "PyTorch"]
DoSkills = ["Linux", "Docker", "Kubernetes", "AWS", "Azure", "CI/CD", "Jenkins", "Terraform", "Ansible"]

patterns = []   

for skill in SweSkills:
    patterns.append({"label": "SWE", "pattern": skill})
for skill in DsSkills:
    patterns.append({"label": "DATASCIENCE", "pattern": skill})
for skill in DoSkills:
    patterns.append({"label": "DEVOPS", "pattern": skill})

ruler.add_patterns(patterns)

doc = nlp("Hello I love me some Angular some python, some other shit too like Terraform, DEVOPS, and idk some other shit")

resume_text = data['']

for ent in doc.ents:
    print(ent.text, ent.label_)

Angular SWE
Terraform ORG
DEVOPS ORG
